<a href="https://colab.research.google.com/github/rbdus0715/Machine-Learning/blob/main/study/cuda/01.intro-cuda/03.hello_cuda_world.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **(1) cuda setting**

In [1]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0


In [2]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git

  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-29pmzahl
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-29pmzahl
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit 0a71d56e5dce3ff1f0dd2c47c29367629262f527
  Preparing metadata (setup.py) ... done
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4295 sha256=f651a20aaab998c9bb68eee46e9112549065a5adbfa599e1879e787526bb3170
  Stored in directory: /tmp/pip-ephem-wheel-cache-gq_oikn4/wheels/a8/b9/18/23f8ef71ceb0f63297dd1903aedd067e6243a68ea756d6feea
Successfully built NVCCPlugin


In [3]:
%load_ext nvcc_plugin

created output directory at /content/src
Out bin /content/result.out


### **(2) CUDA program**
CUDA program의 단계
1. cpu의 초기 데이터를 gpu로 가져온다
2. 커널을 실행하여 전송된 데이터에 대한 작업 수행
3. 그 후 병렬 실행 결과를 cpu 컨텍스트로 다시 전송
4. cpu와 gpu 실행을 위해 할당된 메모리를 시스템에 회수

CUDA program의 요소
- Host code : cpu에서 실행
- Device code : gpu에서 실행

### **(3) hello cuda world 실습**
```cpp
__host__ // host에서 호출 가능한 code
__device__ // device에서 호출 가능한 code
__global__ // host에서 gpu를 이용하기 위해 호출하는 code (code -> kernel)
```

In [4]:
%load_ext nvcc_plugin

The nvcc_plugin extension is already loaded. To reload it, use:
  %reload_ext nvcc_plugin


**- hello cuda world**

In [5]:
%%cu
#include <cuda_runtime.h>
#include <device_launch_parameters.h>
#include <stdio.h>

__global__ void hello_cuda() {
    printf("hello CUDA world \n");
}

int main() {
    hello_cuda <<<1, 20 >>>();
    cudaDeviceSynchronize();

    cudaDeviceReset();
    return 0;
}

hello CUDA world 
hello CUDA world 
hello CUDA world 
hello CUDA world 
hello CUDA world 
hello CUDA world 
hello CUDA world 
hello CUDA world 
hello CUDA world 
hello CUDA world 
hello CUDA world 
hello CUDA world 
hello CUDA world 
hello CUDA world 
hello CUDA world 
hello CUDA world 
hello CUDA world 
hello CUDA world 
hello CUDA world 
hello CUDA world 



**- vector 연산 구현**
- device 메모리를 위한 api (할당, 회수)
```cpp
cudaError_t cudaMalloc(void ** ptr, size_t size) // 동적 할당
cudaError_t cudaFree(void* ptr) // 동적 할당 해제
```
- host, device 간의 데이터 복사
```cpp
// dst: destination 메모리 주소
// src: source 메모리 주소
// count: 복사할 byte 수
// kind: 데이터 전송 타입, ex)host to device, device to host
cudaMemcpy(void* dst, const void* src, size_t size, enum cudaMemcpyKind kind)
// kind에 들어갈 값
// cudaMemcpyHostToHost : cpu에서 cpu로 복사
// cudaMemcpyHostToDevice : cpu에서 gpu로 복사
// cudaMemcpyDeviceToHost : gpu에서 cpu로 복사
// cudaMemcpyDeviceToDevice : gpu에서 gpu로 복사
// cudaMemcpyDefault
```

In [12]:
%%cu
#include <cuda_runtime.h>
#include <device_launch_parameters.h>
#include <stdio.h>
#include <stdlib.h>
#include <string.h>

#define NUM_DATA 512

__global__ void vecAdd(int *_a, int *_b, int *_c) {
    int tID = threadIdx.x;
    _c[tID] = _a[tID] + _b[tID];
}

int main() {

    // 1.
    // host 메모리와 device 메모리가 독립적이기 때문에 따로따로 변수를 선언해주고 복사
    int *a, *b, *c;
    int *d_a, *d_b, *d_c;

    int memSize = sizeof(int)*NUM_DATA;
    printf("%d elements, memSize = %d bytes\n", NUM_DATA, memSize);


    // 2.
    // host 메모리에 데이터 세팅
    a = new int[NUM_DATA]; memset(a, 0, memSize);
    b = new int[NUM_DATA]; memset(b, 0, memSize);
    c = new int[NUM_DATA]; memset(c, 0, memSize);

    for(int i=0; i<NUM_DATA; i++) {
        a[i] = rand() % 10;
        b[i] = rand() % 10;
    }


    // 3.
    // device 메모리를 할당하고, cudaMemcpy를 통해 host 데이터를 device로 넘겨준다.
    cudaMalloc(&d_a, memSize);
    cudaMalloc(&d_b, memSize);
    cudaMalloc(&d_c, memSize);

    cudaMemcpy(d_a, a, memSize, cudaMemcpyHostToDevice);
    cudaMemcpy(d_b, b, memSize, cudaMemcpyHostToDevice);


    // 4.
    // 커널 호출
    vecAdd <<<1, NUM_DATA>>>(d_a, d_b, d_c);

    cudaMemcpy(c, d_c, memSize, cudaMemcpyDeviceToHost);


    // 5.
    // 결과 확인
    bool result = true;
    for(int i=0; i<NUM_DATA; i++) {
        if((a[i] + b[i]) != c[i]) {
            printf("[%d] the results is not matched! (%d, %d)\n", i, a[i]+b[i], c[i]);
            result = false;
        }
    }

    if(result) printf("gpu works well!\n");

    cudaFree(d_a);
    cudaFree(d_b);
    cudaFree(d_c);

    delete[] a;
    delete[] b;
    delete[] c;
}

512 elements, memSize = 2048 bytes
gpu works well!



### **(4) CUDA 단위**
- 스레드가 모여 블럭이 되고, 블럭이 모여 그리드가 된다.
- thread -> warp -> block -> grid
- 커널 함수에서의 kernel<<<a, b>>>
    - Kernel_name<<<**number_of_blocks**, **thread_per_block**>>>(arguments)
    - 쉽게 말해 a개의 블록이 각각 b개의 스레드를 가지고 있다는 뜻

- 다차원 그리드와 블럭 표현하기 위한 dim3 변수
    - dim3 variable_name(X, Y, Z)
    - .x .y .z 를 통해 접근 가능

- *예시 문제*
    - 32개의 스레드가 배열된 1차원 그리드로 커널 실행
    - 각 블록에 x차원으로 4개의 스레드가 있는 8개 스레드 블록
```cpp
dim3 block(4, 1, 1), dim3 grid(8, 1, 1);
Kernel_name<<<grid, block>>>();
```
    
- block size의 한계 범위 : $x \leq 1024, y \leq 1024, z \leq 64$
- block당 스레드 개수 : $x \times  y \times z \leq 1024$ (그래픽  카드 종류에 따라 512개일수도)
- grid :
    - X 차원에 대해 최대 $1^{32}-1$ 개의 스레드 블럭
    - Y, Z 차원에 대해 최대 65536 개의 스레드 블럭


In [7]:
%%cu
#include <cuda_runtime.h>
#include <device_launch_parameters.h>
#include <stdio.h>

__global__ void hello_cuda() {
    printf("hello CUDA world \n");
}

int main() {
    dim3 block(4);
    dim3 grid(8);

    hello_cuda <<<grid, block>>>();
    cudaDeviceSynchronize();

    cudaDeviceReset();
    return 0;
}

hello CUDA world 
hello CUDA world 
hello CUDA world 
hello CUDA world 
hello CUDA world 
hello CUDA world 
hello CUDA world 
hello CUDA world 
hello CUDA world 
hello CUDA world 
hello CUDA world 
hello CUDA world 
hello CUDA world 
hello CUDA world 
hello CUDA world 
hello CUDA world 
hello CUDA world 
hello CUDA world 
hello CUDA world 
hello CUDA world 
hello CUDA world 
hello CUDA world 
hello CUDA world 
hello CUDA world 
hello CUDA world 
hello CUDA world 
hello CUDA world 
hello CUDA world 
hello CUDA world 
hello CUDA world 
hello CUDA world 
hello CUDA world 



In [8]:
%%cu
#include <cuda_runtime.h>
#include <device_launch_parameters.h>
#include <stdio.h>

__global__ void hello_cuda() {
    printf("hello CUDA world \n");
}

int main() {
    int nx, ny;
    nx = 16;
    ny = 4;

    // hello world를 64번 호출하게 됨
    dim3 block(8, 2, 1);
    dim3 grid(nx / block.x, ny / block.y, 1);

    hello_cuda <<<grid, block>>>();
    cudaDeviceSynchronize();

    cudaDeviceReset();
    return 0;
}

hello CUDA world 
hello CUDA world 
hello CUDA world 
hello CUDA world 
hello CUDA world 
hello CUDA world 
hello CUDA world 
hello CUDA world 
hello CUDA world 
hello CUDA world 
hello CUDA world 
hello CUDA world 
hello CUDA world 
hello CUDA world 
hello CUDA world 
hello CUDA world 
hello CUDA world 
hello CUDA world 
hello CUDA world 
hello CUDA world 
hello CUDA world 
hello CUDA world 
hello CUDA world 
hello CUDA world 
hello CUDA world 
hello CUDA world 
hello CUDA world 
hello CUDA world 
hello CUDA world 
hello CUDA world 
hello CUDA world 
hello CUDA world 
hello CUDA world 
hello CUDA world 
hello CUDA world 
hello CUDA world 
hello CUDA world 
hello CUDA world 
hello CUDA world 
hello CUDA world 
hello CUDA world 
hello CUDA world 
hello CUDA world 
hello CUDA world 
hello CUDA world 
hello CUDA world 
hello CUDA world 
hello CUDA world 
hello CUDA world 
hello CUDA world 
hello CUDA world 
hello CUDA world 
hello CUDA world 
hello CUDA world 
hello CUDA world 
hello CUDA